In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [4]:
df_sub=pd.read_csv("sample_submission.csv")
df_train1=pd.read_csv("train.csv")
df_test=pd.read_csv("test.csv")

In [5]:
display(df_train1.shape)
display(df_test.shape)

(22730, 18)

(15154, 17)

# Train Data Cleansing

In [6]:
# droping columns as they are not useful for the current analysis task
df_train = df_train1.copy()
df_train.drop(['id'], axis=1, inplace=True)
df_train.drop(['cityCode'], axis=1, inplace=True)

In [7]:
display(df_train.isnull().sum()/df_train.shape[0]*100)

# droping rows for which any column has NAN values 
df_train.dropna(inplace=True)
display(df_train.info())

squareMeters         0.0
numberOfRooms        0.0
hasYard              0.0
hasPool              0.0
floors               0.0
cityPartRange        0.0
numPrevOwners        0.0
made                 0.0
isNewBuilt           0.0
hasStormProtector    0.0
basement             0.0
attic                0.0
garage               0.0
hasStorageRoom       0.0
hasGuestRoom         0.0
price                0.0
dtype: float64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22730 entries, 0 to 22729
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   squareMeters       22730 non-null  int64  
 1   numberOfRooms      22730 non-null  int64  
 2   hasYard            22730 non-null  int64  
 3   hasPool            22730 non-null  int64  
 4   floors             22730 non-null  int64  
 5   cityPartRange      22730 non-null  int64  
 6   numPrevOwners      22730 non-null  int64  
 7   made               22730 non-null  int64  
 8   isNewBuilt         22730 non-null  int64  
 9   hasStormProtector  22730 non-null  int64  
 10  basement           22730 non-null  int64  
 11  attic              22730 non-null  int64  
 12  garage             22730 non-null  int64  
 13  hasStorageRoom     22730 non-null  int64  
 14  hasGuestRoom       22730 non-null  int64  
 15  price              22730 non-null  float64
dtypes: float64(1), int64(1

None

In [25]:
for cols in df_train:
    print(f'--------------Unique values in column {cols} are: ')
    display(df_train[cols].unique())

--------------Unique values in column squareMeters are: 


array([34291, 95145, 92661, ..., 32138, 13073, 55825], dtype=int64)

--------------Unique values in column numberOfRooms are: 


array([ 24,  60,  45,  99, 100,  36,  95,   3,  84,  35,  22,  86,  49,
        41,  81,  26,  80,  76,  13,  46,  12,  78,  37,  74,  79,  15,
        85,   6,  19,  28,  30,  98,  32,  96,   8,  34,  20,   4,  73,
        48,   2,  58,  42,  59,  39,  69,  75,  62,  89,  21,   9,  29,
        88,  94,  38,  70,  71,  11,  55,  25,  50,  52,  17,  56,  77,
        87,  16,  64,  44,   7,  54,  72,  33,  18,  57,  47,  93,   5,
        61,  90,  83,  10,  51,  68,  27,  97,  31,  92,  67,  66,  63,
        14,  65,  43,  53,   1,  91,  82,  40,  23], dtype=int64)

--------------Unique values in column hasYard are: 


...

array([0, 1], dtype=int64)

--------------Unique values in column hasGuestRoom are: 


array([ 3,  6,  9,  7,  0,  5,  1,  8,  2,  4, 10], dtype=int64)

--------------Unique values in column price are: 


array([3436795.2, 9519958. , 9276448.1, ..., 3661032. , 1316580.1,
       8231424.8])

In [8]:
# x,y splitting
x=df_train.iloc[:,:15]
ay=df_train.price 

# train test splitting
train_X, test_X, train_y, test_y = train_test_split(x, ay,test_size = 0.3, random_state =1234)

In [9]:
model=XGBRegressor(max_depth=3, learning_rate=0.24 , n_estimators=2000, objective='reg:linear', booster='gbtree')
XGB=model.fit(train_X,train_y)
prediction=XGB.predict(test_X)

[16:56:49] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


# Test Data Cleansing 

In [10]:
display(df_test)
df_test.drop(['id','cityCode'], axis=1, inplace=True)

,id,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom
0,22730,47580,89,0,1,8,54830,5,3,1995,0,0,6885,8181,241,0,8
1,22731,62083,38,0,0,87,8576,10,3,1994,1,1,4601,9237,393,1,4
2,22732,90499,75,1,1,37,62454,9,6,1997,0,1,7454,2680,305,0,2
3,22733,16354,47,1,1,9,9262,6,5,2019,1,1,705,5097,122,1,5
4,22734,67510,8,0,0,55,24112,3,7,2014,1,1,3715,7979,401,1,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15149,37879,86184,74,1,1,73,14009,2,10,1993,1,0,3714,3894,665,1,1
15150,37880,45885,41,1,1,64,84208,4,5,2008,0,0,783,7628,976,0,0
15151,37881,73554,51,0,0,41,49263,10,5,2004,0,1,8207,1777,775,0,10
15152,37882,77604,51,0,1,33,67056,4,3,2019,0,0,9578,9706,135,0,2


In [11]:
display(df_test.isnull().sum()/df_test.shape[0]*100)

# droping rows for which any column has NAN values 
df_test.dropna(inplace=True)
display(df_test.info())

squareMeters         0.0
numberOfRooms        0.0
hasYard              0.0
hasPool              0.0
floors               0.0
cityPartRange        0.0
numPrevOwners        0.0
made                 0.0
isNewBuilt           0.0
hasStormProtector    0.0
basement             0.0
attic                0.0
garage               0.0
hasStorageRoom       0.0
hasGuestRoom         0.0
dtype: float64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15154 entries, 0 to 15153
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   squareMeters       15154 non-null  int64
 1   numberOfRooms      15154 non-null  int64
 2   hasYard            15154 non-null  int64
 3   hasPool            15154 non-null  int64
 4   floors             15154 non-null  int64
 5   cityPartRange      15154 non-null  int64
 6   numPrevOwners      15154 non-null  int64
 7   made               15154 non-null  int64
 8   isNewBuilt         15154 non-null  int64
 9   hasStormProtector  15154 non-null  int64
 10  basement           15154 non-null  int64
 11  attic              15154 non-null  int64
 12  garage             15154 non-null  int64
 13  hasStorageRoom     15154 non-null  int64
 14  hasGuestRoom       15154 non-null  int64
dtypes: int64(15)
memory usage: 1.7 MB


None

In [12]:
p_y_train = XGB.predict(x)
display(p_y_train)
score = np.sqrt(mean_squared_error(p_y_train, ay))
display(score)

array([3435591., 9526749., 9281795., ..., 9319794., 6571004., 8230825.],
      dtype=float32)

117559.82095913005

In [15]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission.head()

,id,price
0,22730,4634456.897
1,22731,4634456.897
2,22732,4634456.897
3,22733,4634456.897
4,22734,4634456.897


In [16]:
p_y_test = XGB.predict(df_test)
display(p_y_test)
score = np.sqrt(mean_squared_error(p_y_test, sample_submission.price))
display(score)

array([4758177.5, 6217414. , 9066546. , ..., 7369258.5, 7755354. ,
       1811063.1], dtype=float32)

2918355.074138298

In [17]:
display(df_train1)

,id,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,0,34291,24,1,0,47,35693,2,1,2000,0,1,8,5196,369,0,3,3436795.2
1,1,95145,60,0,1,60,34773,1,4,2000,0,1,729,4496,277,0,6,9519958.0
2,2,92661,45,1,1,62,45457,4,8,2020,1,1,7473,8953,245,1,9,9276448.1
3,3,97184,99,0,0,59,15113,1,1,2000,0,1,6424,8522,256,1,9,9725732.2
4,4,61752,100,0,0,57,64245,8,4,2018,1,0,7151,2786,863,0,7,6181908.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22725,22725,55825,84,1,0,70,12031,3,10,2000,0,0,4477,786,345,0,0,5594137.1
22726,22726,65870,88,1,0,49,23197,9,9,2015,0,1,4811,2454,755,0,7,6594705.0
22727,22727,93192,42,1,0,39,8539,10,5,2014,1,0,5595,4072,789,0,0,9321511.4
22728,22728,65797,86,1,0,89,23197,2,10,2000,1,0,5358,2513,411,0,0,6584708.2


In [18]:
p_y_test = pd.DataFrame(p_y_test)
p_y_test.insert(loc = 0,
          column = 'id',
          value = sample_submission.id)

p_y_test.rename(columns = {0:'price'}, inplace = True)
p_y_test.to_csv('submission.csv', index = False)
pd.read_csv('submission.csv')

,id,price
0,22730,4758177.5
1,22731,6217414.0
2,22732,9066546.0
3,22733,1613400.6
4,22734,6758353.5
...,...,...
15149,37879,8663614.0
15150,37880,4579239.0
15151,37881,7369258.5
15152,37882,7755354.0
